In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
%matplotlib inline

In [ ]:
# Config
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

In [ ]:
# Connect to MIMIC
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)
# cur.close()
# con.close()

In [ ]:
# Load admissions
query = """
SELECT ad.subject_id
  , ad.hadm_id
  , ad.admission_type
  , ad.diagnosis
  , ic.icustay_id
  , ic.first_careunit
  , ic.last_careunit
  , ic.intime as icu_intime
  , ad.hospital_expire_flag
  , pa.expire_flag
FROM admissions ad
INNER JOIN icustays ic
ON ad.subject_id = ic.subject_id
INNER JOIN patients pa
ON ad.subject_id = pa.subject_id
WHERE ad.subject_id = 40080
"""

admissions = pd.read_sql_query(query,con)

In [ ]:
admissions.head()

In [ ]:
# Load chartevents
query = """
SELECT ce.icustay_id
  , ce.charttime
  , di.label
  , ce.value
  , ce.valuenum
  , ce.valueuom
FROM chartevents ce
INNER join d_items di
ON ce.itemid = di.itemid
INNER join icustays ie
ON ce.icustay_id = ie.icustay_id
WHERE ce.icustay_id = 252522
AND error != 1
ORDER BY ce.charttime
"""

charts = pd.read_sql_query(query,con)

In [ ]:
# Run the query and assign the results to a variable
charts.head()

In [ ]:
# Load outputevents
query = """
select oe.icustay_id
  , oe.charttime
  , di.label
  , oe.value
  , oe.valueuom
from outputevents oe 
inner join icustays ie
  on oe.icustay_id = ie.icustay_id
inner join d_items di
  on oe.itemid = di.itemid
where oe.subject_id = 40080
order by oe.charttime
"""

outputs = pd.read_sql_query(query,con)

In [ ]:
outputs.head()

In [ ]:
# Load inputevents
query = """
SELECT inp.icustay_id
  , inp.starttime
  , inp.endtime
  , di.label
  , inp.amount
  , inp.amountuom
  , inp.rate
  , inp.rateuom
FROM inputevents_mv inp 
INNER join icustays ie
  ON inp.icustay_id = ie.icustay_id
INNER join d_items di
  ON inp.itemid = di.itemid
WHERE inp.subject_id = 40080
AND lower(inp.statusdescription) != 'rewritten'
ORDER BY inp.starttime
"""

inputs = pd.read_sql_query(query,con)

In [ ]:
inputs.head()

In [ ]:
# Load labevents
query = """
SELECT le.subject_id
  , ie.icustay_id
  , le.charttime
  , di.label
  , le.value
  , le.valuenum
  , le.valueuom
FROM labevents le
INNER join icustays ie
  ON le.subject_id = ie.subject_id
  AND le.charttime >= ie.intime
  AND le.charttime <= ie.outtime
INNER JOIN d_labitems di
  ON le.itemid = di.itemid
where le.subject_id = 40080
"""

labs = pd.read_sql_query(query,con)

In [ ]:
labs.head()

In [ ]:
# Plot data over first 24 hours from admission to ICU
# admissions
# charts
# outputs
# inputs
# labs

